In [ ]:
import torch
import torch.nn as nn

In [ ]:
class Discriminator(nn.Module):
    def __init__(self,channel_img,feature_d,num_classes,img_size):
        super().__init__()
        self.img_size=img_size
        self.disc=nn.Sequential(
                nn.Conv2d(channel_img+1,feature_d,kernel_size=4,stride=2,padding=1),
                nn.LeakyReLU(0.2),
                self._block(feature_d,feature_d*2,4,2,1),
                self._block(feature_d*2,feature_d*4,4,2,1),
                self._block(feature_d*4,feature_d*8,4,2,1),
                nn.Conv2d(feature_d*8,1,4,2,0))
        self.embed=nn.Embedding(num_classes,img_size*img_size)
        
    def _block(self,in_channel,out_channel,kernel,stride,padding):
        return nn.Sequential(
                nn.Conv2d(in_channel,out_channel,kernel,stride,padding),
                nn.InstanceNorm2d(out_channel),
                 nn.LeakyReLU(0.1),)
    
    def forward(self,x,labels):
        embedding=self.embed(labels).view(labels.shape[0],1,self.img_size,self.img_size)
        x=torch.cat([x,embedding],dim=1)
        return self.disc(x)